In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import jupyros
import rospy
import math
from turtlesim.srv import Spawn
from turtlesim.msg import Pose
from time import time

In [ ]:
rospy.init_node("superturtle")

# Turtlesim

In [ ]:
# !rosrun turtlesim turtlesim_node
# !rosrun turtlesim turtle_teleop_key

In [ ]:
turtlesim = jupyros.TurtleWidget()
display(turtlesim.canvas)

In [ ]:
# Canvas default size is 1600x1600 starting from top-left corner
turtlesim.spawn(name="simon", pose={"x": 600, "y": 350, "theta": 20})

In [ ]:
# Turtles need a certain amount of time to spawn
turtlesim.spawn(pose={"x": 1050, "y": 1250, "theta": 90})

In [ ]:
# Change the turtle path color
turtlesim.turtles["turtle1"].path_color = "#BF0059"

### Subscriber

In [ ]:
topic_name = '/turtle1/pose'

# Based on the dimensions of window controlled by teleop keyboard
def convert_xy_units(x, y):
    convert_factor = 11.08
    x_convert = x / convert_factor * turtlesim.canvas.width
    y_convert = (convert_factor - y) / convert_factor * turtlesim.canvas.height
    
    return x_convert, y_convert

def callback_move_turtle(msg):
    elapsed_time = time() - turtlesim.last_move_time

    if elapsed_time > 0.1:
        x, y = convert_xy_units(msg.x, msg.y)
        turtlesim.move_to_pose(name="turtle1", 
                               new_pose={"x": x, 
                                         "y": y, 
                                         "theta": msg.theta})

jupyros.subscribe(topic_name, Pose, lambda msg: callback_move_turtle(msg))

### Spiral Publisher

In [ ]:
%%thread_cell

rate = rospy.Rate(5)
pub = rospy.Publisher(topic_name, Pose, queue_size=10, latch=True)
for i in range(360*5):
    msg = Pose()
    msg.x = (i/1800*5)*math.sin(i / 180 * math.pi) + 11.08 / 2
    msg.y = (i/1800*5)*math.cos(i / 180 * math.pi) + 11.08 / 2
    msg.theta = - i / 180 * math.pi
    pub.publish(msg)
    rate.sleep()

## Spawn Service and Client

In [ ]:
def spawn_turtle(msg):
    print(f"Spawning turtle at x={msg.x} and y={msg.y}")
    turtlesim.spawn(name=msg.name, 
                    pose={"x": msg.x, 
                          "y": msg.y, 
                          "theta": msg.theta})
    return "Spawned"

In [ ]:
%%thread_cell

srv = rospy.Service('spawn8', Spawn, spawn_turtle)

In [ ]:
jupyros.client('spawn8', Spawn)

In [ ]:
print(turtlesim.name_index.keys())

# tf2 Demo

In [ ]:
turtles_tf2 = jupyros.TurtleWidget()
display(turtles_tf2.canvas)

In [ ]:
turtles_tf2.spawn(name="turtle2", new_pose={"x": 700,
                                            "y": 1150,
                                            "theta": math.radians(90)})

In [ ]:
topic1 = '/turtle1/pose'
topic2 = '/turtle2/pose'

def callback_move_turtle(msg, turtle_name):
    elapsed_time = time() - turtles_tf2.last_move_time

    if elapsed_time > 0.1:
        x, y = convert_xy_units(msg.x, msg.y)
        pose = {"x": x, "y": y, "theta": msg.theta}
        turtles_tf2.move_to_pose(name=turtle_name, new_pose=pose)

In [ ]:
jupyros.subscribe(topic1, Pose, 
                  lambda msg: callback_move_turtle(msg, turtle_name="turtle1"))

In [ ]:
jupyros.subscribe(topic2, Pose, 
                  lambda msg: callback_move_turtle(msg, turtle_name="turtle2"))